## 연습 문제 1. 영화 추천하기

데이터 셋 출처: https://www.kaggle.com/datasets/hijest/genre-classification-dataset-imdb

IMDB는 영화 리뷰를 제공하는 웹 서비스입니다. 이 사이트에서 크롤링한 영화 제목, 장르, 줄거리 데이터 셋을 가지고 간단한 추천을 구현해보고자 합니다. 사용자는 자연어 형태로 본인이 원하는 영화의 장르, 특징, 줄거리 등을 입력하면 tf-idf를 이용하여 적절한 영화를 추천해주는 시스템을 개발해보세요.

In [36]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()
df = pd.read_csv("./data/movies_preprocessed.csv")
df["all_tokens"] = df["all_tokens"].progress_apply(lambda x: eval(x))
df['genre'].unique


100%|█████████████████████████████████████████████████████████████████████████| 54214/54214 [00:04<00:00, 13282.86it/s]


<bound method Series.unique of 0              drama
1           thriller
2              adult
3              drama
4              drama
            ...     
54209         comedy
54210         horror
54211    documentary
54212         comedy
54213        history
Name: genre, Length: 54214, dtype: object>

In [4]:
print(df.iloc[0])

title                              Oscar et la dame rose (2009)
genre                                                     drama
content       Listening in to a conversation between his doc...
all           Oscar et la dame rose drama Listening in to a ...
all_tokens    ['oscar', 'et', 'la', 'dame', 'rose', 'drama',...
Name: 0, dtype: object


In [5]:
from util import preprocess

preprocess("Listening to the music")

['listen', 'music']

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 더미 함수 정의: 입력을 그대로 반환
def dummy_fun(doc):
    return doc

# TF-IDF Vectorizer 객체 생성
tfidf = TfidfVectorizer(
    analyzer='word', 
    tokenizer=dummy_fun,
    preprocessor=dummy_fun, 
    token_pattern=None
)

# all_tokens 열로부터 TF-IDF 벡터를 학습 및 생성
tfidf_csr_matrix = tfidf.fit_transform(df['all_tokens'])

# TF-IDF 벡터의 형태 및 유형 출력
print(tfidf_csr_matrix.shape)
print(type(tfidf_csr_matrix))



(54214, 116)
<class 'scipy.sparse._csr.csr_matrix'>


In [30]:
def search(query, k=5):
    query = " ".join(preprocess(query)) # 입력을 전처리하고 리스트를 문자열로 변환
    query_tfidf = tfidf.transform([query]) # 전처리된 쿼리를 TF-IDF 벡터로 변환
    similarities = cosine_similarity(query_tfidf, tfidf_csr_matrix).flatten() # 변환된 쿼리와 모든 영화 간의 코사인 유사도를 계산
    top_similarities = sorted(similarities)[-k:][::-1] # 상위 k개의 가장 유사한 문서의 유사도를 가져옴
    top_indices = similarities.argsort()[-k:][::-1] # 상위 k개의 가장 유사한 문서의 인덱스를 가져옴
    top_titles = [df.iloc[i]['title'] for i in top_indices] # 상위 k개의 문서의 제목을 가져옴
    for similarity, title in zip(top_similarities, top_titles):
        print(round(similarity, 4), title) # 상위 k개의 문서의 유사도와 제목을 출력

        
        
# 사용 예시:
search("I want to watch a comedy movie") # 사용자가 코미디 영화를 보고 싶어한다고 가정


0.5885 International Comedian (2017)
0.5725 Maggie (1960)
0.5718 Kaffeefahrt ins Krematorium (2011)
0.5667 Voices from the Graves (2006)
0.5659 Gay USA (1978)


In [31]:
# 사용 예시:
search("I want to watch a sci-fi movie") # 사용자가 코미디 영화를 보고 싶어한다고 가정

0.558 Taking Tiger Mountain (1983)
0.5548 Voices from the Graves (2006)
0.553 A Big Tit Christmas 2 (2011)
0.5516 The Kitchenistas of National City (2015)
0.5512 In Cockpit (2008)


In [34]:
# 사용 예시:
search("I want to watch a action movie") # 사용자가 코미디 영화를 보고 싶어한다고 가정

0.5815 AMAZONIA 'Out of the Dark' (????)
0.575 International Comedian (2017)
0.5661 The Kitchenistas of National City (2015)
0.5615 Kirn's reconciliation (2017)
0.5571 The Lost Civilizations of North America (2010)


In [35]:
# 사용 예시:
search("I want to watch a romance movie") # 사용자가 코미디 영화를 보고 싶어한다고 가정

0.5881 International Comedian (2017)
0.5716 AMAZONIA 'Out of the Dark' (????)
0.5634 Transnational Tradeswomen (2006)
0.5605 Kaffeefahrt ins Krematorium (2011)
0.5585 Ocean (2017/III)


In [21]:
df['genre'].unique()

array(['drama', 'thriller', 'adult', 'documentary', 'comedy', 'crime',
       'reality-tv', 'horror', 'sport', 'animation', 'action', 'fantasy',
       'short', 'sci-fi', 'music', 'adventure', 'talk-show', 'western',
       'family', 'mystery', 'history', 'news', 'biography', 'romance',
       'game-show', 'musical', 'war'], dtype=object)

In [24]:
search("I want to watch a sci-fi ")


0.8366 Mary McCarthy's Paris (1964)
0.8341 The Ebb-Tide (1998)
0.8325 The Right Hand Man (2004)
0.8287 Hein Fach (2004)
0.8264 Checking Out (1989)
